In [1]:
!pip install transformers[torch] evaluate rouge_score  wandb --upgrade
!pip install -U datasets

  Obtaining dependency information for transformers[torch] from https://files.pythonhosted.org/packages/12/dd/f17b11a93a9ca27728e12512d167eb1281c151c4c6881d3ab59eb58f4127/transformers-4.35.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 6.2 MB/s eta 0:00:00
  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 40.8 MB/s eta 0:00:0000:0100:01
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=f472af8b94b9ab46b1472ac35498a941f502a2862a5c7e0f022085d5a3a36ee4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_sco

In [2]:
!pip install pandas==1.5.3
!pip install datasets==2.8.0
!pip install fsspec==2023.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 54.1 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
beatrix-jupyterlab 2023.814.150030 requires jupyter-server~=1.16, but you have jupyter-server 2.10.0 which is incompatible.
beatrix-jupyterlab 2023.814.150030 requires jupyterlab~=3.4, but you have jupyterlab 4.0.5 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you ha

In [3]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=bf2eb6d8eff0edff00aaf21788ee5a34c837856be85cfe954fca0e90bbccb0af
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil


In [4]:
!pip install gdown
!gdown --id 1MKEMgJ0wiRHgm9W0eVtQlhtwryhh4yXU

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1MKEMgJ0wiRHgm9W0eVtQlhtwryhh4yXU
To: /kaggle/working/train.csv
100%|███████████████████████████████████████| 11.1M/11.1M [00:00<00:00, 204MB/s]


In [5]:
import random, os

import pandas as pd
import numpy as np
import json

from tqdm._tqdm_notebook import tqdm

import torch
from transformers import (AutoTokenizer,
                          DataCollatorForSeq2Seq,
                          AutoModelForSeq2SeqLM,
                          Seq2SeqTrainingArguments,
                          Seq2SeqTrainer,
                          pipeline,
                          EarlyStoppingCallback)

import evaluate

from datasets import load_dataset

/tmp/ipykernel_47/403521310.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
import wandb
wandb.login() # d1772ceadec86b483bc945f4429fb0a6902ff72d

%env WANDB_PROJECT=meeting_summarization_sweep
%env WANDB_LOG_MODEL=true

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=meeting_summarization_sweep
env: WANDB_LOG_MODEL=true


In [7]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(42)

In [8]:
# TRAIN_SOURCE = os.path.join("./train.json")
# TEST_SOURCE = os.path.join("./test.json")

In [9]:
# with open(TRAIN_SOURCE) as f:
#     TRAIN_DATA = json.loads(f.read())

# with open(TEST_SOURCE) as f:
#     TEST_DATA = json.loads(f.read())

In [10]:
# TRAIN_DATA[0]

In [11]:
# train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
# uid = 1000
# for data in TRAIN_DATA:
#     for agenda in data['context'].keys():
#         context = ''
#         for line in data['context'][agenda]:
#             context += data['context'][agenda][line]
#             context += ' '
#         train.loc[uid, 'uid'] = uid
#         train.loc[uid, 'title'] = data['title']
#         train.loc[uid, 'region'] = data['region']
#         train.loc[uid, 'context'] = context[:-1]
#         train.loc[uid, 'summary'] = data['label'][agenda]['summary']
#         uid += 1


# test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
# uid = 2000
# for data in TEST_DATA:
#     for agenda in data['context'].keys():
#         context = ''
#         for line in data['context'][agenda]:
#             context += data['context'][agenda][line]
#             context += ' '
#         test.loc[uid, 'uid'] = uid
#         test.loc[uid, 'title'] = data['title']
#         test.loc[uid, 'region'] = data['region']
#         test.loc[uid, 'context'] = context[:-1]
#         uid += 1

In [12]:
# train['text'] = train.title + ' ' + train.region + ' ' + train.context
# test['text'] = test.title + ' ' + test.region + ' ' + test.context

In [13]:
# train = train.drop(['title','region','context'], axis=1)
# train.head()

In [14]:
# test = test.drop(['title','region','context'], axis=1)
# test.head()

In [15]:
# train.to_csv('./train.csv', index=False)
# test.to_csv('./test.csv', index=False)

In [16]:
raw_dataset =  load_dataset('csv', data_files='./train.csv',split='train')
split_data = raw_dataset.train_test_split(.5)
dataset = split_data['test'].train_test_split(.2)
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:727: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-7827b7dba44f3b67/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['uid', 'summary', 'text'],
        num_rows: 1197
    })
    test: Dataset({
        features: ['uid', 'summary', 'text'],
        num_rows: 300
    })
})

In [17]:
dataset['train'][0]

{'uid': 1141,
 'summary': '음성군의회 제210회 제2차 정례회 회기는 11월 30일부터 12월 28일까지 29일간으로 가결됨.',
 'text': '제210회    본회의 제1차(2009.11.30.) 음성 의사일정 제1항, 음성군의회 제210회 제2차 정례회 회기결정의 건을 상정합니다. 음성군의회 제210회 제2차 정례회 회기는 「음성군의회 회기와 그 운영 등에 관한 조례」제3조제2항의 규정 및 의원님들께서 사전에 협의하여 주신 것과 같이 11월 30일부터 12월 28일까지 29일간으로 하고자 합니다. 그리고 주요 의사일정으로는 군정질문 및 답변과 행정사무감사 실시와 2010년도 당초예산안 심사 등이 주요 의사일정으로 배부하여 드린 유인물과 같이 의사일정을 결정하고자 하는데, 의원 여러분! 이의가 없으신지요? (「없습니다」하는 의원 있음) 이의가 없으므로 가결되었음을 선포합니다.'}

In [18]:
checkpoint = "eenzeenee/t5-small-korean-summarization"
# checkpoint = 'hyunwoongko/kobart'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [19]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [20]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [22]:
rouge = evaluate.load("rouge")

In [23]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True )
  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  result['gen_len'] = np.mean(prediction_lens)

  return {k: round(v, 4) for k, v in result.items()}


In [24]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/780 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/383M [00:00<?, ?B/s]

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu' )
# LEARNING_RATE = 2e-4
# BATCH_SIZE = 8
# WEIGHT_DECAY = 0.01
# EPOCHS = 4

In [26]:
def model_init():
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
    return model

In [27]:
# confit default and sweep config
num_epochs = 5
Batch_size = 16
Learning_rate = 2e-4
Scheduler_type = "linear"
Weight_decay = 0.0

config_defaults ={
    'epochs' : num_epochs,
    'batch_size':Batch_size,
    'learning_rate':Learning_rate,
    'weight_decay':Weight_decay,
    'optimizer' :'AdamW',
    'SchedulerType': Scheduler_type
}

sweep_config ={
    'method':'random', #'grid', 'random', 'bayes'
}

parameters_dict = {
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'max':2e-1,
        'min': 2e-7
    },
    'weight_decay':  {
        'values': [0.01, 0.03, 0.1, 0.3, 0.5]
    },
    'SchedulerType':{
        'values': ["linear", "cosine", "constant"]
    },
}

sweep_config['parameters'] = parameters_dict

In [28]:
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache() 

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  9% |  2% |


In [29]:
def train(config=None):
    with wandb.init(config= config_defaults):
        config = wandb.config # set sweep configuration
        
    training_args = Seq2SeqTrainingArguments(
        output_dir="meeting_transcript",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        learning_rate= config.learning_rate,
        per_device_train_batch_size=Batch_size,
        per_device_eval_batch_size=Batch_size,
        weight_decay=config.weight_decay,
        num_train_epochs=num_epochs,
        lr_scheduler_type = config.SchedulerType,
        predict_with_generate=True,
        fp16=True,
        gradient_checkpointing=True,
        gradient_accumulation_steps = 8,
        report_to="wandb",
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
#         model_init=model_init,
    )

    trainer.train()



In [30]:
# def model_init(trial):
#     return AutoModelForSequenceClassification.from_pretrained(
#         model_args.checkpoint,
#         from_tf=bool(".ckpt" in model_args.checkpoint),
#         config=config,
#         cache_dir=model_args.cache_dir,
#         revision=model_args.model_revision,
#         use_auth_token=True if model_args.use_auth_token else None,
#     )

In [31]:
entity = "solowosophia13"
sweep_id = wandb.sweep(sweep_config, entity= entity, project="meeting_summarization_sweep")
wandb.agent(sweep_id, train, count=30)

Create sweep with ID: fn6xs74k
Sweep URL: https://wandb.ai/solowosophia13/meeting_summarization_sweep/sweeps/fn6xs74k


wandb: Agent Starting Run: 1z7scneq with config:
wandb: 	SchedulerType: linear
wandb: 	learning_rate: 1.0228568142716974e-05
wandb: 	weight_decay: 0.5
wandb: Currently logged in as: solowosophia13. Use `wandb login --relogin` to force relogin


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,No log,1.472888,0.386300,0.181000,0.384100,0.382200,58.026700
1,No log,1.360138,0.415100,0.206900,0.415300,0.412200,56.196700
2,No log,1.290980,0.433300,0.229400,0.432800,0.430600,55.636700
3,No log,1.260535,0.452700,0.249200,0.453500,0.449800,56.246700
4,No log,1.251727,0.455100,0.250300,0.455900,0.452400,56.590000


...

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,No log,9.411661,0.000000,0.000000,0.000000,0.000000,127.000000
1,No log,9.411661,0.000000,0.000000,0.000000,0.000000,127.000000
2,No log,9.411661,0.000000,0.000000,0.000000,0.000000,127.000000
3,No log,9.411661,0.000000,0.000000,0.000000,0.000000,127.000000
4,No log,9.411661,0.000000,0.000000,0.000000,0.000000,127.000000


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


eval/gen_len,▁▁▁▁▁
eval/loss,▁▁▁▁▁
eval/rouge1,▁▁▁▁▁
eval/rouge2,▁▁▁▁▁
eval/rougeL,▁▁▁▁▁
eval/rougeLsum,▁▁▁▁▁
eval/runtime,▃▅█▅▁
eval/samples_per_second,▆▄▁▄█
eval/steps_per_second,▆▅▁▅█
train/epoch,▁▃▅▆██
train/global_step,▁▃▅▆██
